In [1]:
import pandas as pd

In [2]:
compute_samples = pd.read_csv('../sourmash/maca/facs_compute_v7_all/samples.csv')
compute_samples.head()

,id,read1,read2,name,output,ksizes,protein,scaled,trim_low_abundance_kmers
0,A1-B000126-3_39_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...,cell_ontology_class:epidermal_cell|tissue:Skin...,s3://olgabot-maca/facs/sourmash_compute_all/A1...,27,True,1000,True
1,A1-B000127-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...,cell_ontology_class:endothelial_cell|tissue:Fa...,s3://olgabot-maca/facs/sourmash_compute_all/A1...,27,True,1000,True
2,A1-B000167-3_56_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...,cell_ontology_class:basal_cell|tissue:Mammary_...,s3://olgabot-maca/facs/sourmash_compute_all/A1...,27,True,1000,True
3,A1-B000168-3_57_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_00...,s3://czbiohub-maca/remux_data/170925_A00111_00...,A1-B000168-3_57_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1...,27,True,1000,True
4,A1-B000412-3_56_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_00...,s3://czbiohub-maca/remux_data/170914_A00111_00...,cell_ontology_class:endothelial_cell|tissue:He...,s3://olgabot-maca/facs/sourmash_compute_all/A1...,27,True,1000,True


In [5]:
import os

signatures_dir = compute_samples['output'].map(os.path.dirname).unique()[0]
signatures_dir

's3://olgabot-maca/facs/sourmash_compute_all'

In [9]:
ksizes = compute_samples['ksizes'].unique()
ksizes

array([27])

In [8]:
sequences_to_compare = ['protein', 'dna']

In [10]:
import itertools

In [18]:
output = 's3://olgabot-maca/facs/sourmash_index_all/'

In [23]:
pd.options.display.max_colwidth = 100

In [27]:
data = list(itertools.product(sequences_to_compare, ksizes))

columns = ['sequence_to_compare', 'ksize']

samples = pd.DataFrame(data, columns=columns)

samples['signatures'] = signatures_dir
samples['id'] = samples.apply(lambda x: 'k{ksize}_{sequence_to_compare}'.format(**x), axis=1)
samples['output'] = samples.apply(lambda x: output + '{id}.sbt.json'.format(**x), axis=1)
samples = samples.set_index('id')
# sapmles['ksize'] = 
samples.head()

,sequence_to_compare,ksize,signatures,output
id,,,,
k27_protein,protein,27,s3://olgabot-maca/facs/sourmash_compute_all,s3://olgabot-maca/facs/sourmash_index_all/k27_protein.sbt.json
k27_dna,dna,27,s3://olgabot-maca/facs/sourmash_compute_all,s3://olgabot-maca/facs/sourmash_index_all/k27_dna.sbt.json


In [29]:
import json

folder = '../sourmash/maca/facs_index_v7_all'

! mkdir -p $folder
samples.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": "../../../reflow/sourmash_index.rf",
		"runs_file": "samples.csv"
	}

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! head -n 3 $folder/samples.csv $folder/config.json

==> ../sourmash/maca/facs_index_v7_all/samples.csv <==
id,sequence_to_compare,ksize,signatures,output
k27_protein,protein,27,s3://olgabot-maca/facs/sourmash_compute_all,s3://olgabot-maca/facs/sourmash_index_all/k27_protein.sbt.json
k27_dna,dna,27,s3://olgabot-maca/facs/sourmash_compute_all,s3://olgabot-maca/facs/sourmash_index_all/k27_dna.sbt.json

==> ../sourmash/maca/facs_index_v7_all/config.json <==
{"program": "../../../reflow/sourmash_index.rf", "runs_file": "samples.csv"}